In [ ]:
import os
import sys
from pathlib import Path

import torch
from PIL import Image 
from loguru import logger
from omegaconf import OmegaConf

project_root = Path().resolve().parent
sys.path.append(str(project_root))

from yolo import (
    AugmentationComposer, 
    bbox_nms, 
    create_model, 
    custom_logger, 
    create_converter,
    draw_bboxes, 
    Vec2Box
)
from yolo.config.config import NMSConfig

In [ ]:
MODEL = "v9-c"
DEVICE = "cuda:0"

WEIGHT_PATH = f"../weights/{MODEL}.pt" 
TRT_WEIGHT_PATH = f"../weights/{MODEL}.trt"
MODEL_CONFIG = f"../yolo/config/model/{MODEL}.yaml"

IMAGE_PATH = "../demo/images/inference/image.png"
IMAGE_SIZE = (640, 640)

custom_logger()
device = torch.device(DEVICE)
image = Image.open(IMAGE_PATH)

In [ ]:
with open(MODEL_CONFIG) as stream:
    cfg_model = OmegaConf.load(stream)
if os.path.exists(TRT_WEIGHT_PATH):
    from torch2trt import TRTModule

    model_trt = TRTModule()
    model_trt.load_state_dict(torch.load(TRT_WEIGHT_PATH))
else:
    from torch2trt import torch2trt


    model = create_model(cfg_model, weight_path=WEIGHT_PATH)
    model = model.to(device).eval()

    dummy_input = torch.ones((1, 3, 640, 640)).to(device)
    logger.info(f"♻️ Creating TensorRT model")
    model_trt = torch2trt(model, [dummy_input])
    torch.save(model_trt.state_dict(), TRT_WEIGHT_PATH)
    logger.info(f"📥 TensorRT model saved to oonx.pt")

transform = AugmentationComposer([], IMAGE_SIZE)
converter = create_converter(cfg_model.name, model_trt, cfg_model.anchor, IMAGE_SIZE, device)


In [ ]:
image, bbox, rev_tensor = transform(image, torch.zeros(0, 5))
image = image.to(device)[None]

In [ ]:
with torch.no_grad():
    predict = model_trt(image)
    predict = converter(predict["Main"])
predict_box = bbox_nms(predict[0], predict[2], NMSConfig(0.5, 0.5))
draw_bboxes(image, predict_box)

Sample Output:

![image](../demo/images/output/visualize.png)